# Exam - 2 feb 2022

## Part 1 - MCQ

[...]

## Part 2 - Exercises

### Ex 1

In [ ]:
input_actions = './Actions.txt'
input_apps = './Apps.txt'
out_path = '/user/s315054/exam_sim/2022-02-04_1'

actions_DF = spark.read.load(input_actions, format='csv', header=False, inferSchema=True)\
                    .toDF('userID', 'appID', 'timestamp', 'action')
actions_DF.createOrReplaceTempView('actions_table')

apps_DF = spark.read.load(input_apps, format='csv', header=False, inferSchema=True)\
                    .toDF('appID', 'appname', 'price', 'category', 'commpany')
apps_DF.createOrReplaceTempView('apps_table')

spark.udf.register('getYear', lambda ts: int(ts.split('/')[0]))

# Group by user and app, having downloaded the app at least once
out_DF = spark.sql("""
            SELECT appID, count(distinct(userID))
            FROM actions_table, apps_table
            WHERE actions_table.appID == apps_table.appID AND apps_table.price == 0 AND actions_table == 'Install'
            GROUP BY appID
            HAVING count(distinct(userID)) > 1000000
""")

# Join on the free apps to isolate them
out_DF = selected_DF.join(apps_DF.filter('price == 0').select('appID'), 'appID')

out_DF.write.csv(out_path, sep=',', header=False)


### Ex 2.1


In [ ]:
out_path_2a = 'outFolder_ex2_1'

# Categories with average price > 10 euri
apps_DF = spark.read.load(input_apps, format='csv', header=False, inferSchema=True)\
                    .toDF('appID', 'appname', 'price', 'category', 'commpany')
apps_DF.createOrReplaceTempView('apps_table')

out_2a_DF = spark.sql("""
            SELECT category
            FROM apps_table
            WHERE price > 0
            GROUP BY category
            HAVING avg(price) > 10
""")



###  Ex 2.2

In [ ]:
out_path_2b = 'outFolder_ex2_2'

actions_DF = spark.read.load(input_actions, format='csv', header=False, inferSchema=True)\
                    .toDF('userID', 'appID', 'timestamp', 'action')

actions_DF.createOrReplaceTempView('act')

spark.udf.register('getYear', lambda ts: int(ts.split('/')[0]))

# Isolate apps not intalled before 2021
# Also check for n. intalls from 2021 > 0 in case there are records of 'downloaded' from 2021 on, but not installed
new_apps_DF = spark.sql("""
        SELECT appID
        FROM act
        WHERE action == 'Installed'
        GROUP BY appID
        HAVING sum(CAST(getYear(timestamp) <= 2020 AS int)) == 0 AND sum(CAST(getYear(timestamp) >= 2021 AS int)) > 0
""")
new_apps_DF.createOrReplaceTempView('new_apps')

# Keep apps downloaded from 2021 on
# Keep users which have them installed
installed_DF = spark.sql("""
        SELECT act.appID
        FROM act, new_apps
        WHERE act.appID == new_apps.appID
        GROUP BY act.appID, act.userID
        HAVING sum(CAST(act.action == 'Install' AS int)) > sum(CAST(act.action == 'Remove' AS int))
""")

intalled_DF.createOrReplaceTempView('installed')

out_2_2_DF = spark.sql("""
        SELECT appID
        FROM installed
        GROUP BY appID
        HAVING count(userID) >= 100000
""")

out_2_2_DF.write.csv(out_path_2b, header=False)

In [ ]:
# This returns the couples appID, userID for which the app was not downloaded
# NO - there could be apps which have been downloaded before 2021 by some users and not by others.
# These apps would have been included in the query 
# (because of the users which did not download them prior to 2021)
spark.sql("""
        SELECT appID, userID
        FROM act
        GROUP BY appID, userID
        HAVING sum(CAST((getYear(timestamp) <= 2020 AND action == 'Installed') AS int)) == 0 AND sum(CAST((getYear(timestamp) >= 2021 AND action == 'Installed') AS int)) > 0
""")

